# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [33]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [34]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [35]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [36]:
def image_cropping(image):
    #h,w,c = image.shape
    #starth = h//2-(croph//2)
    #startw = w//2-(cropw//2)
    if image.shape[0] > 100 and image.shape[1] > 100:
        no_rows,no_cols,no_chs=image.shape
        if no_rows == 120 and no_cols == 160:
            image = image[0:120,20:140]
        elif no_rows == 360 and no_cols == 360:
            image = image[0:360:3,0:360:3]
            
    return image

In [37]:
from skimage.transform import rescale
def normalize_img(img):
    img = (img - np.min(img))/(np.max(img) - np.min(img))
    #img = rescale(img, 0.25, multichannel=True, mode='constant')
    return img

In [38]:
img_idx = [n for n in range(0,30,2)]
#img_idx = [n for n in range(30)]
#img_idx = [n for n in range(5,25,1)]

img_row = 120
img_col = 120


In [39]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    print('length of folder list = ', len(folder_list))
    
    #img_idx = [0,3,5,7,10,13,15,18,20,23,27] #create a list of image numbers you want to use for a particular video
    #img_idx = [7,8,9,10,11,12,13,14,15,19,20,21,22,23,24]
    x = len(img_idx)
    y = img_row
    z = img_col

    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #print("before :", image.shape)
                    #image = crop_center(image, y, z)
                    #image = np.resize(image, (y, z, 3))
                    #print("after :", image.shape)
                    
                    image = image_cropping(image)
                    
                    batch_data[folder,idx,:,:,0] = normalize_img(image)[:, :, 0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_img(image)[:, :, 1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_img(image)[:, :, 2] #normalise and feed in the image
                    
                    #batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                    #batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    #batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_data = len(folder_list) % batch_size
        num_batch = len(folder_list)//batch_size # calculate the number of batches
        if (remaining_data > 0) :
            batch_data = np.zeros((remaining_data,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_data,5)) # batch_labels is the one hot representation of the output
            for folder in range(remaining_data): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #print("before :", image.shape)
                    #image = crop_center(image, y, z)
                    #image = np.resize(image, (y, z, 3))
                    #print("after :", image.shape)
                    
                    image = image_cropping(image)
                    
                    batch_data[folder,idx,:,:,0] = normalize_img(image)[:, :, 0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_img(image)[:, :, 1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_img(image)[:, :, 2] #normalise and feed in the image
                    
                    #batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                    #batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    #batch_data[folder,idx,:,:,2] = image[:,:,2]/255

                    
                batch_labels[folder, int(t[folder + (num_batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


In [40]:
def generator_cnn(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    print('length of folder list = ', len(folder_list))
    
    #img_idx = [0,3,5,7,10,13,15,18,20,23,27] #create a list of image numbers you want to use for a particular video
    #img_idx = [7,8,9,10,11,12,13,14,15,19,20,21,22,23,24]
    x = len(img_idx)
    y = img_row
    z = img_col

    while True:
        t = np.random.permutation(folder_list)
        num_batches = (len(folder_list))//batch_size  # calculate the number of batches
        num_img = (batch_size * x)                
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((num_img,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #print("before :", image.shape)
                    #image = crop_center(image, y, z)
                    #image = np.resize(image, (y, z, 3))
                    #print("after :", image.shape)
                    
                    image = image_cropping(image)
                    
                    batch_itr = idx + (folder * x)
                    batch_data[batch_itr,:,:,0] = normalize_img(image)[:, :, 0] #normalise and feed in the image
                    batch_data[batch_itr,:,:,1] = normalize_img(image)[:, :, 1] #normalise and feed in the image
                    batch_data[batch_itr,:,:,2] = normalize_img(image)[:, :, 2] #normalise and feed in the image
                    
                    #batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                    #batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    #batch_data[folder,idx,:,:,2] = image[:,:,2]/255
                    
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_data = len(folder_list) % batch_size
        num_batch = len(folder_list)//batch_size # calculate the number of batches
        if (remaining_data > 0) :
            batch_data = np.zeros((remaining_data*x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_data,5)) # batch_labels is the one hot representation of the output
            for folder in range(remaining_data): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #print("before :", image.shape)
                    #image = crop_center(image, y, z)
                    #image = np.resize(image, (y, z, 3))
                    #print("after :", image.shape)
                    
                    image = image_cropping(image)
                    
                    batch_itr = idx + (folder * x)   
                    batch_data[batch_itr,:,:,0] = normalize_img(image)[:, :, 0] #normalise and feed in the image
                    batch_data[batch_itr,:,:,1] = normalize_img(image)[:, :, 1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_img(image)[:, :, 2] #normalise and feed in the image
                    
                    #batch_data[folder,idx,:,:,0] = image[:,:,0]/255
                    #batch_data[folder,idx,:,:,1] = image[:,:,1]/255
                    #batch_data[folder,idx,:,:,2] = image[:,:,2]/255

                    
                batch_labels[folder, int(t[folder + (num_batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [41]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [42]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, Reshape
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

#img_rows = 100
#img_cols = 100
input_shape = (len(img_idx), img_row, img_col, 3)
input_shape1 = (img_row, img_col, 3)


In [43]:
#write your model here
#Model1
#model = Sequential()
#model.add(Conv3D(32, kernel_size=(3, 3, 1),strides = (1,1,1), padding = 'same', activation='relu',input_shape=input_shape, data_format = 'channels_last'))
#model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Conv3D(64, kernel_size=(3, 3, 1),activation='relu'))
#model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dense(5, activation='softmax'))


In [44]:
#write your model here
#model 2
#model = Sequential()
#model.add(Conv3D(32, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu',input_shape=input_shape, data_format = 'channels_last'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))
#model.add(Conv3D(32, kernel_size=(3, 3, 3), activation = 'relu'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))
#model.add(Conv3D(32, kernel_size=(3, 3, 3), activation = 'relu'))
#model.add(Conv3D(32, kernel_size=(3, 3, 3), activation = 'relu'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))
#model.add(Conv3D(64, kernel_size=(2, 2, 2), activation = 'relu'))
#model.add(Conv3D(64, kernel_size=(2, 2, 2), activation = 'relu'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2), strides = (1,2,2)))

#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(5, activation='softmax'))


In [45]:
#write your model here
#model 3
#model = Sequential()
#model.add(Conv3D(32, kernel_size=(2, 2, 2),strides=(1, 1, 1), padding = 'sa#me', activation = 'relu',input_shape=input_shape, data_for#mat = 'channels_last'))
#model.add(#maxPooling3D(pool_size=(1, 2, 2)))
#model.add(Conv3D(32, kernel_size=(2, 2, 2), activation = 'relu'))
#model.add(#maxPooling3D(pool_size=(1, 2, 2)))
#model.add(Conv3D(32, kernel_size=(2, 2, 2), activation = 'relu'))
#model.add(Conv3D(32, kernel_size=(2, 2, 2), activation = 'relu'))
#model.add(#maxPooling3D(pool_size=(1, 2, 2)))
#model.add(Conv3D(64, kernel_size=(2, 2, 2), activation = 'relu'))
#model.add(Conv3D(64, kernel_size=(2, 2, 2), activation = 'relu'))
#model.add(#maxPooling3D(pool_size=(1, 2, 2)))

#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(5, activation='soft#max'))


In [46]:
#write your model here
#model 4
#model = Sequential()
#model.add(Conv3D(8, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu',input_shape=input_shape, data_format = 'channels_last'))
#model.add(Conv3D(16, kernel_size=(3, 3, 3), activation = 'relu'))
#model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Conv3D(32, kernel_size=(3, 3, 3), activation = 'relu'))
#model.add(Conv3D(64, kernel_size=(3, 3, 3), activation = 'relu'))
#model.add(MaxPooling3D(pool_size=(2, 2, 1)))
#model.add(Conv3D(64, kernel_size=(1, 1, 1), activation = 'relu'))

#model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(5, activation='softmax'))


In [47]:
#write your model here
#model 5
model = Sequential()
model.add(Conv3D(8, kernel_size=(3, 3, 3), padding = 'same', activation = 'relu',input_shape=input_shape, data_format = 'channels_last'))
model.add(Conv3D(16, kernel_size=(3, 3, 3), activation = 'relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation = 'relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation = 'relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation = 'relu'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))


In [48]:
##### write code here
#model 6 with CNN + GRU
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3,3), padding='same', activation = 'relu', input_shape=input_shape1))
cnn_model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
cnn_model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
print(cnn_model.output_shape)
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.2))
print(cnn_model.output_shape)

#l = cnn_model.layers[len(cnn_model.layers)-1]
new_shape1 = (cnn_model.output_shape[1]* cnn_model.output_shape[2], cnn_model.output_shape[3])
print(new_shape1)
cnn_model.add(Reshape(new_shape1))
print(cnn_model.output_shape)
#model.add(GRU(120, return_sequences=True, input_shape=new_shape))
cnn_model.add(GRU(64, return_sequences=True, activation='relu' ))
print(cnn_model.output_shape)
cnn_model.add(Reshape((cnn_model.output_shape[1],cnn_model.output_shape[2], 1) ))
cnn_model.add(TimeDistributed(Dense(1)))

cnn_model.add(Flatten())
cnn_model.add(Dense(5, activation='softmax'))


(None, 27, 27, 128)
(None, 27, 27, 256)
(729, 256)
(None, 729, 256)
(None, 729, 64)


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [49]:
optimiser = optimizers.SGD(lr=0.01) #write your optimizer
#optimiser = optimizers.Adam()
#optimiser = Adadelta(lr=0.1)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

cnn_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (cnn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 15, 120, 120, 8)   656       
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 13, 118, 118, 16)  3472      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 13, 59, 59, 16)    0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 11, 57, 57, 32)    13856     
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 9, 55, 55, 64)     55360     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 4, 27, 27, 64)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 2, 25, 25, 64)     110656    
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [50]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

train_generator_cnn = generator_cnn(train_path, train_doc, batch_size)
val_generator_cnn = generator_cnn(val_path, val_doc, batch_size)

In [51]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001, cooldown=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [52]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 32
length of folder list =  100
Source path =  ./Project_data/train ; batch size = 32
length of folder list =  663
Epoch 1/15


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


21/21 [==============================] - 111s 5s/step - loss: 1.5975 - categorical_accuracy: 0.2287 - val_loss: 1.5645 - val_categorical_accuracy: 0.3300

Epoch 00001: saving model to model_init_2019-09-2219_14_49.234678/model-00001-1.59778-0.22775-1.56448-0.33000.h5
Epoch 2/15
20/21 [===========================>..] - ETA: 1s - loss: 1.5666 - categorical_accuracy: 0.2687

In [ ]:
cnn_model.fit_generator(train_generator_cnn, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator_cnn, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)